In [ ]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup

import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

Opções disponiveis:
 - Brasileirão Serie A  [2023 a 2014]
 - Premier League       [2023 a 2014]  *2014 significa a temporada 2015-2014*

In [31]:
ligas = {'brasileirao_a': ["Série A", "https://fbref.com/en/comps/24/Serie-A-Stats", 'Brasileirao', 'https://fbref.com/en/comps/24/schedule/Serie-A-Scores-and-Fixtures'],
         'premier_league': ["Premier League", "https://fbref.com/en/comps/9/Premier-League-Stats", 'Premier_League', 'https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures'],}

anos = list(range(2023, 2013, -1))

season_atual = 2023
liga_atual = ligas['brasileirao_a']

Liga Selecionada: \
Anos Selecionados: [2023 a 2014]\
Season Atual: 2023

# Funcoes

In [ ]:
def match_history(league, years = [2023], update = None):
  print("Getting the seasons: ", years)
  match_history = []
  for year in years:
    print("Scraping the season: ", year)
    data = requests.get(league[1])
    soup = BeautifulSoup(data.text)
    tabela = soup.select('table.stats_table')[0]                        #Seleciona a tabela principal
    links = tabela.find_all('a')                            #Procura a Anchor que contem todos os links do time
    links = [link.get('href') for link in links]            #Pega os links (sem o começo deles)
    links = [link for link in links if '/squads/' in link]  #Pega apenas o link 'squads'
    urls = [f"https://fbref.com{link}" for link in links]   #Adiciona o inicio do html

    try:
      prev_season = soup.select('a.prev')[0].get('href')      #Vai para a temporada anterior
      league[1] = f"https://fbref.com{prev_season}"
    except IndexError:
      continue

    #Itera sobre todos os times da tabela
    for team in urls:
      nome_time = team.split('/')[-1].replace('-Stats', '').replace('-','_').lower()
      data = requests.get(team)
      soup = BeautifulSoup(data.text)

      ##Partidas jogadas
      matches = pd.read_html(data.text, match= 'Scores & Fixtures')[0]

      ##Chutes
      links = [link.get("href") for link in soup.find_all('a')]
      links = [link for link in links if link and 'all_comps/shooting/' in link]
      data_shooting = requests.get(f"https://fbref.com{links[0]}")
      shooting = pd.read_html(data_shooting.text, match= "Shooting")[0]
      shooting.columns = shooting.columns.droplevel()

      ##Goleiros
      #links = [l.get("href") for l in soup.find_all('a')]
      #links = [l for l in links if l and 'all_comps/keeper' in l]
      #data_goalkeeping = requests.get(f"https://fbref.com{links[0]}")
      #goalkeeping = pd.read_html(data_goalkeeping.text, match= "Goalkeeping")[0]
      #goalkeeping.columns = goalkeeping.columns.droplevel()

      ##Passes
      #links = [l.get("href") for l in soup.find_all('a')]
      #links = [l for l in links if l and 'all_comps/passing' in l]
      #data_passing = requests.get(f"https://fbref.com{links[0]}")
      #passing = pd.read_html(data_passing.text, match= "Passing")[0]
      #passing.columns = passing.columns.droplevel()

      ##Tipos de Passes
      #links = [l.get("href") for l in soup.find_all('a')]
      #links = [l for l in links if l and 'all_comps/passing_types' in l]
      #data_passtype = requests.get(f"https://fbref.com{links[0]}")
      #pass_types = pd.read_html(data_passtype.text, match= "Pass Types")[0]
      #pass_types.columns = pass_types.columns.droplevel()

      ##Gols e Criação de Chutes
      #links = [l.get("href") for l in soup.find_all('a')]
      #links = [l for l in links if l and 'all_comps/gca' in l]
      #data_shotcreation = requests.get(f"https://fbref.com{links[0]}")
      #goal_shotcreation = pd.read_html(data_shotcreation.text, match= "Goal and Shot Creation")[0]
      #goal_shotcreation.columns = goal_shotcreation.columns.droplevel()

      ##Acoes Defensivas
      #links = [l.get("href") for l in soup.find_all('a')]
      #links = [l for l in links if l and 'all_comps/defense' in l]
      #data_defensive = requests.get(f"https://fbref.com{links[0]}")
      #defensive = pd.read_html(data_defensive.text, match= "Defensive Actions")[0]
      #defensive.columns = defensive.columns.droplevel()

      ##Posse de Bola
      #links = [l.get("href") for l in soup.find_all('a')]
      #links = [l for l in links if l and 'all_comps/possession' in l]
      #data_possession = requests.get(f"https://fbref.com{links[0]}")
      #possession = pd.read_html(data_possession.text, match= "Possession")[0]
      #possession.columns = possession.columns.droplevel()

      ##Demais Estatisticas
      #links = [l.get("href") for l in soup.find_all('a')]
      #links = [l for l in links if l and 'all_comps/misc' in l]
      #data_misc = requests.get(f"https://fbref.com{links[0]}")
      #misc = pd.read_html(data_misc.text, match= "Miscellaneous Stats")[0]
      #misc.columns = misc.columns.droplevel()


      # Merge das abas
      try:
        matches_df = matches.merge(
          shooting[['Date', 'Sh', 'SoT']], on= 'Date')
        #matches_df = matches_df.merge(
          #goalkeeping[['Date', ]], on= 'Date')
        #matches_df = matches_df.merge(
          #passing[['Date', ]], on= 'Date') 
        #matches_df = matches_df.merge(
          #pass_types[['Date', ]], on= 'Date')
        #matches_df = matches_df.merge(
          #goal_shotcreation[['Date', ]], on= 'Date')
        #matches_df = matches_df.merge(
          #defensive[['Date', ]], on= 'Date')
        #time_df = time_df.merge(
          #possession[['Date', ]], on= 'Date')
        #time_df = time_df.merge(
          #miscellaneous[['Date',]], on= 'Date')
      except ValueError:
        continue

      matches_df = matches_df[matches_df["Comp"] == league[0]]
      matches_df['Season'] = year
      matches_df['Team'] = nome_time
      
      if update is None:
        match_history.append(matches_df)
      else:
        update.drop(update[update['season'] == year].index, inplace= True)
        update.append(matches_df)
        match_history = update

      time.sleep(10)
    
  if update is None:
    match_history = pd.concat(match_history)
  match_history.columns = [c.lower() for c in match_history.columns]

  return match_history

In [ ]:
def standings(league, years= [2023], update = False):
  table = {}
  print("Getting the seasons: ", years)
  for year in years:
    print("Scraping the season: ", year)
    data = requests.get(league[1])
    soup = BeautifulSoup(data.text)
    table[year] = pd.read_html(data.text, match= 'Regular season')[0]
  
    try:
      prev_season = soup.select('a.prev')[0].get('href')      #Vai para a temporada anterior
      league[1] = f"https://fbref.com{prev_season}"
    except IndexError:
      continue
    

  return table

In [30]:
def next_matches(league):
  data = requests.get(league[3])
  matches = pd.read_html(data.text, match= 'Scores & Fixtures')[0]
  return matches

# Exports

### Baixar anos anteriores:

In [ ]:
# historico = match_history(liga_atual, years= anos)
# historico.to_excel(f'datasets/{liga_atual[2]}/match_history/historico-2015a2023(naotratada).xlsx', index= False)

In [ ]:
# tabela = standings(liga_atual, years= anos)
# for index, season in enumerate(tabela):
#  tabela[season].to_excel(f'datasets/{liga_atual[2]}/standings/tabela-{season}(naotratada).xlsx', index=False)

### Atualizar temporada atual:

In [ ]:
historico = pd.read_excel(f'datasets/{liga_atual[2]}/match_history/historico-2015a2023(naotratada).xlsx')

historico = match_history(liga_atual, update= historico)

historico.to_excel(f'datasets/{liga_atual[2]}/match_history/historico-2015a2023(naotratada).xlsx', index= False)

In [28]:
tabela = standings(liga_atual, update= True)[season_atual]

tabela.to_excel(f'datasets/{liga_atual[2]}/standings/tabela-2023(naotratada).xlsx', index= False)

Getting the seasons:  [2023]
Scraping the season:  2023


In [32]:
rodadas = next_matches(liga_atual)

rodadas.to_excel(f'datasets/{liga_atual[2]}/rounds/prox_partidas(naotratada).xlsx', index=False)